In [ ]:
!pip install -q sentence-transformers langchain  chromadb accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 xformers

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.agents.agent_types import AgentType
from langchain.agents import create_csv_agent

In [ ]:
model_name = "NousResearch/llama-2-7b-chat-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_name)

base_model = LlamaForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map='auto',
)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    max_length=4096,
    temperature=1,
    top_p=0.95,
    repetition_penalty=1.15
)
llm = HuggingFacePipeline(pipeline=pipe)

## csv_agent

In [ ]:
csv_path = "/content/YourFILE.csv"
from langchain.agents.agent_types import AgentType

from langchain.agents import create_csv_agent
agent = create_csv_agent(llm,
                         csv_path,
                         verbose=True ,
                         agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
                         )

In [ ]:
agent.run("how many rows are there?")

In [ ]:
agent.run("how many columns are there?")

## CSVLoader

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="/content/YourFILE.csv")
data = loader.load()

In [ ]:
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
embed_model = HuggingFaceEmbeddings(model_name=embed_model_id)

In [ ]:
from langchain.vectorstores import Chroma
vectordb = Chroma.from_documents(data, embedding=embed_model , persist_directory="DB")

In [ ]:
from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(llm=llm , retriever=vectordb.as_retriever())

In [ ]:
chain("ASK About Your Data")